In [1]:

import os
import time
import oss2
import json

import cv2 as cv
import numpy as np
from tqdm import tqdm
from scipy.optimize import minimize, basinhopping

from aliyunsdkcore.client import AcsClient
from aliyunsdkcore.acs_exception.exceptions import ClientException
from aliyunsdkcore.acs_exception.exceptions import ServerException
from aliyunsdkfacebody.request.v20191230.CompareFaceRequest import CompareFaceRequest


In [2]:

access_key_id = os.getenv('OSS_TEST_ACCESS_KEY_ID', '')
access_key_secret = os.getenv('OSS_TEST_ACCESS_KEY_SECRET', '')
bucket_name = os.getenv('OSS_TEST_BUCKET', 'ivan-bucket-out-002')
endpoint = os.getenv('OSS_TEST_ENDPOINT', 'oss-cn-shanghai.aliyuncs.com')

bucket = oss2.Bucket(oss2.Auth(access_key_id, access_key_secret), endpoint, bucket_name)

client = AcsClient(access_key_id, access_key_secret, 'cn-shanghai')


In [3]:

#
def _f1(imgA=None, imgB=None):
    while True:
        try:
            time.sleep(0.68)
            if imgA:
                oss2.resumable_upload(bucket, 'imgA.jpg', imgA)
            if imgB:
                oss2.resumable_upload(bucket, 'imgB.jpg', imgB)

            request = CompareFaceRequest()
            request.set_accept_format('json')

            request.set_ImageURLA(f"https://{bucket_name}.{endpoint}/imgA.jpg")
            request.set_ImageURLB(f"https://{bucket_name}.{endpoint}/imgB.jpg")

            response = client.do_action_with_exception(request)
            return json.loads(response)["Data"]["Confidence"]
        except Exception as e:
            print(e)

#
def _f2(imgA, imgB):
    imgA, imgB = cv.imread(imgA), cv.imread(imgB)
    return np.sqrt(np.sum(np.square(imgA - imgB)))


In [4]:
%time _f1(imgA='../Data/Z0.jpg', imgB='../Data/Z0.jpg')

CPU times: user 18.3 ms, sys: 4.43 ms, total: 22.8 ms
Wall time: 1.08 s


100.0

In [5]:
%time _f1(imgA='../Data/Z0.jpg')

CPU times: user 8.47 ms, sys: 1.85 ms, total: 10.3 ms
Wall time: 954 ms


100.0

In [6]:
%time _f1(imgA='../Data/Z0.jpg', imgB='../Data/Z1.jpg')

CPU times: user 11.2 ms, sys: 2.75 ms, total: 14 ms
Wall time: 1.01 s


45.32559585571289

In [7]:
%time _f1(imgA='../Data/bad.png', imgB='../Data/bad.png')

CPU times: user 12.3 ms, sys: 3.77 ms, total: 16.1 ms
Wall time: 1.02 s


0.0

In [8]:
%time _f2(imgA='../Data/Z0.jpg', imgB='../Data/Z1.jpg')

CPU times: user 1.33 ms, sys: 1.32 ms, total: 2.65 ms
Wall time: 1.28 ms


768.1659976853962

In [9]:
%time _f2(imgA='../Data/Z0.jpg', imgB='../Data/Z0.jpg')

CPU times: user 1.38 ms, sys: 1.31 ms, total: 2.68 ms
Wall time: 1.29 ms


0.0

In [10]:

pNum = "T000001"

proj = ".."
pimg = f"{proj}/Data/images"
pmdl = f"{proj}/Model"
pout = f"{proj}/Out/{pNum}"

os.system(f"rm -rf {pout}/temp*")
os.system(f"rm -rf {pout}/images/*")


0

In [11]:
# ! rm -rf ../Out/T000001/images/*

In [12]:

st, ed = 1, 1
_N, M = 128, 128# [2, 4, 8, 16, 32, 64, 128]
pl = lambda x: sorted(
    [
        (i-_N//x, i, j-_N//x, j)
        for j in range(0, _N+1, _N//x) if j > 0
        for i in range(0, _N+1, _N//x) if i > 0
     ]
)
dpl = pl(M)


In [18]:


RUN = 2
BIG = 25
CUT = 50
change = lambda X: np.array([min(max(iX, -BIG), BIG) for iX in X.astype(np.int)])

for i in tqdm(range(st, ed+1), desc="All"):
    mscore = 0
    pi = f"{pimg}/"+f"00000{i}"[-5:]+".jpg"
    op = f"{pout}/images/"+f"00000{i}"[-5:]+".jpg"
    tp = f"{pout}/temp.jpg"
    
    mescore = _f1(imgA=pi, imgB=pi)
    if mescore == 0:# or os.path.exists(op):
        print(pi, "%.6f"%mescore)
        continue
    
    score = {}
    
    # 
    def _function(X):
        X = change(X)
        _X = X.tostring()
        
        if _X in score:
            return score[_X]
        else:
            imgb = cv.imread(pi)
            imgo = imgb.copy()
            for nX, lX in enumerate(X):
                i_group = dpl[nX]
                imgo[i_group[0]:i_group[1], i_group[2]:i_group[3]] = \
                imgo[i_group[0]:i_group[1], i_group[2]:i_group[3]] - lX
            imgo = np.clip(imgo, imgb-BIG, imgb+BIG)
            cv.imwrite(tp, imgo)

            _s1 = _f1(imgB=tp)
            _s2 = _f2(imgA=pi, imgB=tp)

            _s1 = CUT if _s1 <= CUT else _s1
            _s2 = _s2
            _s_ = (2*_s1*_s2)/(_s1+_s2)
            
            score[_X] = _s_
                
            print("%.2f, %.6f, %.6f, %.6f" % (np.mean(np.abs(X)), _s1, _s2, _s_))
            return _s_
    
    try:
        for _ in range(100):
            X0 = np.random.randint(-BIG, BIG+1, size=(M**2))
            _function(X0)
            X = X0
        
        # print("Start.")
        # res = minimize(_function, X0, method="Nelder-Mead", options={"xatol": 1e-1, "disp": True})
        # res = basinhopping(_function, X0, minimizer_kwargs={"method": "BFGS"}, niter=RUN)
        # X = res.x
    except Exception as e:
        print(e)
    
    X = change(X)
    imgb = cv.imread(pi)
    imgo = imgb.copy()
    for nX, lX in enumerate(X):
        i_group = dpl[nX]
        imgo[i_group[0]:i_group[1], i_group[2]:i_group[3]] = \
        imgo[i_group[0]:i_group[1], i_group[2]:i_group[3]] - lX
    imgo = np.clip(imgo, imgb-BIG, imgb+BIG)
    cv.imwrite(f"{op}", imgo)
    print(">>>", pi, "%.6f"%_f1(imgA=f"{pi}", imgB=f"{op}"), "%.6f"%_f2(imgA=f"{pi}", imgB=f"{op}"))




All:   0%|          | 0/1 [00:00<?, ?it/s]

12.87, 51.400394, 1874.512470, 100.057154
12.76, 56.087051, 1879.602086, 108.923832
12.75, 50.000000, 1884.357716, 97.415163
12.76, 50.000000, 1870.702275, 97.396786
12.84, 50.000000, 1879.491155, 97.408643
12.72, 50.000000, 1870.822279, 97.396948
12.76, 50.000000, 1872.192832, 97.398804
12.75, 50.000000, 1868.676269, 97.394037
12.88, 50.000000, 1882.090593, 97.412130
12.79, 50.000000, 1876.740525, 97.404944
12.69, 50.000000, 1869.013376, 97.394494
12.62, 50.000000, 1870.980759, 97.397163
12.76, 50.000000, 1879.534783, 97.408702
12.71, 50.000000, 1875.702002, 97.403544
12.83, 50.000000, 1881.061137, 97.410750
12.77, 50.000000, 1876.473554, 97.404584
12.74, 50.000000, 1875.604969, 97.403413
12.71, 50.000000, 1876.913690, 97.405177
12.74, 50.000000, 1872.759461, 97.399571
12.73, 50.269444, 1878.773802, 97.918918
12.74, 50.000000, 1871.830120, 97.398313
12.72, 50.000000, 1879.704764, 97.408930
12.74, 50.000000, 1877.933705, 97.406550
12.73, 50.000000, 1873.417999, 97.400461
12.66, 50.0000


All: 100%|██████████| 1/1 [01:52<00:00, 112.82s/it]

>>> ../Data/images/00001.jpg 43.879784 1870.496191


In [ ]:

# st, el = 1, 712

# savl = []
# for i in tqdm(range(st, el+1)):
#     pi = f"{pimg}/"+f"00000{i}"[-5:]+".jpg"
#     op = f"{pout}/images/"+f"00000{i}"[-5:]+".jpg"
    
#     if not os.path.exists(op):
#         img = cv.imread(pi)
#         cv.imwrite(op, img)
#         savl.append(1)
# print(len(savl))


In [ ]:
# ! sh run.sh

In [ ]:
# help(minimize)